# 16x16 Sudoku Solver

*by Wei Xi, Yaowei Zong, December, 2017*

## Introduction

Sudoku has been one of the most popular brain game for a long time. The goal of the game is to fill a 9x9 puzzle grid with numbers so that each row, column and 3×3 section contain all of the digits between 1 and 9. Both of us like solving a 9x9 sudoku puzzle at ordinary times but we never tried to solve a 16x16 one. And we are wondering if there’s any AI related algorithm that can help us to do so. After doing some research online, we’ve found that the constrict propagation and the search algorithm is very useful to those kind of problem. So for this term project, we would like to study the algorithms and the 9x9 sudoku solver which is written by Peter Norvig used and modifty it so it can also be applied to a 16x16 sudoku solver.


## Methods

So at first, we tried to solve the 16x16 puzzle only by constraint popagation. In constraint popagation each state is list of valid values for each variable and next states are ones with different list of valid values. We implemented several methods for it. First, we need to read in the puzzle and parse it into a grid. Since it's a 16x16 puzzle, we define columns as hexidecimal number from 1 to 16, so we have "123456789abcdefg" and rows as upper case alphabet numbers, so we have "ABCDEFGHIJKLMNOP". We do a cross product of the cols of row so we can have A1,A2...Pf,Pg represents all the squares. We also define the units and peers. In one unit, all the digit can only appear once. Each square should have 3 unit and 48 peers where the sqaures that shares a unit is call peer.

In [ ]:
def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

In [ ]:
digits   = '123456789abcdefg'
rows     = 'ABCDEFGHIJKLMNOP'
cols     = digits
squares  = cross(rows, cols)
unitlist = ([cross(rows, c) for c in cols] +
        [cross(r, cols) for r in rows] +
        [cross(rs, cs) for rs in ('ABCD','EFGH','IJKL','MNOP') for cs in ('1234','5678','9abc','defg')])
units = dict((s, [u for u in unitlist if s in u]) 
         for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
         for s in squares)
grid1  = '.f.1.2aec........63gc.84ef1.2...e.97b3f.........4d2c....6....f......e1b735a..8.c3g..24...e7d..5fb.5......94..6......d.g5f..c........9.1c.83ab.f.2c.b..e354....9.63.4..d..b91.cg2..a9......c.8.67c8..g..a.d...5..5...3.46.1f......916.e.b..2...a8.e...d9.4cb8..2.'.lower()


A1 A2 A3 A4 |A5 A6 A7 A8 |A9 Aa Ab Ac |Ad Ae Af Ag  
B1 B2 B3 B4 |B5 B6 B7 B8 |B9 Ba Bb Bc |Bd Be Bf Bg  
C1 C2 C3 C4 |C5 C6 C7 C8 |C9 Ca Cb Cc |Cd Ce Cf Cg  
D1 D2 D3 D4 |D5 D6 D7 D8 |D9 Da Db Dc |Dd De Df Dg  
------------+------------+------------+-----------  
E1 E2 E3 E4 |E5 E6 E7 E8 |E9 Ea Eb Ec |Ed Ee Ef Eg  
F1 F2 F3 F4 |F5 F6 F7 F8 |F9 Fa Fb Fc |Fd Fe Ff Fg  
G1 G2 G3 G4 |G5 G6 G7 G8 |G9 Ga Gb Gc |Gd Ge Gf Gg  
H1 H2 H3 H4 |H5 H6 H7 H8 |H9 Ha Hb Hc |Hd He Hf Hg  
------------+------------+------------+-----------  
I1 I2 I3 I4 |I5 I6 I7 I8 |I9 Ia Ib Ic |Id Ie If Ig  
J1 J2 J3 J4 |J5 J6 J7 J8 |J9 Ja Jb Jc |Jd Je Jf Jg  
K1 K2 K3 K4 |K5 K6 K7 K8 |K9 Ka Kb Kc |Kd Ke Kf Kg  
L1 L2 L3 L4 |L5 L6 L7 L8 |L9 La Lb Lc |Ld Le Lf Lg  
------------+------------+------------+-----------  
M1 M2 M3 M4 |M5 M6 M7 M8 |M9 Ma Mb Mc |Md Me Mf Mg  
N1 N2 N3 N4 |N5 N6 N7 N8 |N9 Na Nb Nc |Nd Ne Nf Ng  
O1 O2 O3 O4 |O5 O6 O7 O8 |O9 Oa Ob Oc |Od Oe Of Og  
P1 P2 P3 P4 |P5 A6 A7 A8 |A9 Aa Ab Ac |Ad Ae Af Ag  

Then, we will have parse_grid_16(), grid_value_16(), assign and eliminate() to pass a given grid. For example,'.f.1.2aec........63gc.84ef1.2...e.97b3f.........4d2c....6....f......e1b735a..8.c3g..24...e7d..5fb.5......94..6......d.g5f..c........9.1c.83ab.f.2c.b..e354....9.63.4..d..b91.cg2..a9......c.8.67c8..g..a.d...5..5...3.46.1f......916.e.b..2...a8.e...d9.4cb8..2.'. We convert a given grid into a dict of {square: char} with '0' or '.' for empties. After that, it's time to implement the body of the constraint propagation. There are two main roles it should follow, "(1) If a square has only one possible value, then eliminate that value from the square's peers. (2) If a unit has only one possible place for a value, then put the value there.",quated from Peter Norvig. 


In [ ]:
def parse_grid_16(grid):
	"""Convert grid to a dict of possible values, {square: digits}, or
	return False if a contradiction is detected."""
	## To start, every square can be any digit; then assign values from the grid.
	values = dict((s, digits) for s in squares)
	for s,d in grid_values_16(grid).items():
		if d in digits and not assign(values, s, d):
			return False ## (Fail if we can't assign d to square s.)
	return values
def grid_values_16(grid):
	#"Convert grid into a dict of {square: char} with '0' or '.' for empties."
	chars = [c for c in grid if c in digits or c in '0.']
	assert len(chars) == 256
	return dict(zip(squares, chars))

def assign(values, s, d):
	"""Eliminate all the other values (except d) from values[s] and propagate.
	Return values, except return False if a contradiction is detected."""
	other_values = values[s].replace(d, '')
	if all(eliminate(values, s, d2) for d2 in other_values):
		return values
	else:
		return False

def eliminate(values, s, d):
	"""Eliminate d from values[s]; propagate when values or places <= 2.
	Return values, except return False if a contradiction is detected."""
	if d not in values[s]:
		return values ## Already eliminated
	values[s] = values[s].replace(d,'')
	## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
	if len(values[s]) == 0:
		return False ## Contradiction: removed last value
	elif len(values[s]) == 1:
		d2 = values[s]
		if not all(eliminate(values, s2, d2) for s2 in peers[s]):
			return False
	## (2) If a unit u is reduced to only one place for a value d, then put it there.
	for u in units[s]:
		dplaces = [s for s in u if d in values[s]]
	if len(dplaces) == 0:
		return False ## Contradiction: no place for this value
	elif len(dplaces) == 1:
		# d can only be in one place in unit; assign it there
			if not assign(values, dplaces[0], d):
				return False
	return values



def display_16(values):
	#"Display these values as a 2-D grid."
	width = 1+max(len(values[s]) for s in squares)
	line = '+'.join(['-'*(width*4)]*4)
	for r in rows:
		print (''.join(values[r+c].center(width)+('|' if c in '48c' else '')
					  for c in cols))
		if r in 'DHL': print (line)
	print

Ok, let's see how it works for a simple 16x16 grid.

In [1]:
grid1  = '.f.1.2aec........63gc.84ef1.2...e.97b3f.........4d2c....6....f......e1b735a..8.c3g..24...e7d..5fb.5......94..6......d.g5f..c........9.1c.83ab.f.2c.b..e354....9.63.4..d..b91.cg2..a9......c.8.67c8..g..a.d...5..5...3.46.1f......916.e.b..2...a8.e...d9.4cb8..2.'.lower()
display_16(parse_grid_16(grid1))

NameError: name 'display_16' is not defined

Some square are defined but some are not. It's the time to implement search algorithm.

In [ ]:
def solve_16(grid): return search(parse_grid_16(grid))

def search(values):
	"Using depth-first search and propagation, try all possible values."
	if values is False:
		return False ## Failed earlier
	if all(len(values[s]) == 1 for s in squares): 
		return values ## Solved!
	## Chose the unfilled square s with the fewest possibilities
	n,s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
	return some(search(assign(values.copy(), s, d)) 
		for d in values[s])

def some(seq):
	"Return some element of seq that is true."
	for e in seq:
		if e: return e
	return False

import time
def solve_all(grids, name='', showif=0.0):
	"""Attempt to solve a sequence of grids. Report results.
	When showif is a number of seconds, display puzzles that take longer.
	When showif is None, don't display any puzzles."""
	def time_solve(grid):
		start = time.clock()
		values = solve_16(grid)
		t = time.clock()-start
		## Display puzzles that take long enough
		if showif is not None and t > showif:
			display_16(grid_values_16(grid))
			print("")
			if values: display_16(values)
			print ('(%.2f seconds)\n' % t)
		return (t, solved(values))
	times, results = zip(*[time_solve(grid) for grid in grids])
	N = len(results)
	if N > 1:
		print ("Solved %d of %d %s puzzles (avg %.2f secs (%d Hz), max %.2f secs)." % (
			sum(results), N, name, sum(times)/N, N/sum(times), max(times)))
			


def solved(values):
	"A puzzle is solved if each unit is a permutation of the digits 1 to 9."
	def unitsolved(unit): return set(values[s] for s in unit) == set(digits)
	return values is not False and all(unitsolved(unit) for unit in unitlist)

What we have is a simply depth first search algorithm to try all posibilities. 

In [ ]:
solve_all([grid1])

## Results

Show all results.  Intermediate resultw might be shown in above Methods section.  Plots, tables, whatever.

## Conclusions

What I learned.asdferaf ef erge g erg 

### References

* [Russell and Norvig, 2014] Stuart Russell and Peter Norvig, [Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu/), Publisher. 2014.
* [chuck] me, fldfkafjlak

Your report should contain approximately 1,500 to 5,000 words, in markdown cells.  You can count words by running the following python code in your report directory.

In [2]:
import io
from nbformat import current
import glob
nbfile = glob.glob('*.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Sudoku Solver.ipynb is 762


/usr/local/lib/python3.6/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)
